In [ ]:
# https://en.wikipedia.org/wiki/Big_Five_personality_traits
# https://ipip.ori.org/newBigFive5broadKey.htm

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline

In [ ]:
file_to_open = pd.read_csv("Resources/data.csv", delimiter="\t")
data=file_to_open.copy()
data.head()

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
data.isnull().values.any()

In [ ]:
data.isnull().sum()

In [ ]:
data.dropna(inplace=True)

In [ ]:
print(f"Number of participants: {len(data)}")

In [ ]:
data.age.value_counts()

In [ ]:
# calculate maximum year we're looking for in the data
max_year = 2012-13
print(f"Youngest participient's date of birth: {max_year}")

In [ ]:
data= data.loc[data["age"]<2000]
data.head()

In [ ]:
data["age"] = [2012-x if x >1960 else x for x in data["age"]]
data= data.loc[data["age"]<=120]
data.head()

In [ ]:
data.age.sort_values(ascending=False) 

In [ ]:
data.describe()

In [ ]:
# data.dtypes

In [ ]:
data.corr()

In [ ]:
data.country.value_counts()

In [ ]:
data.country.value_counts().mean()

In [ ]:
countries = pd.DataFrame(data["country"].value_counts())
c_125= countries.loc[countries["country"]>125]
plt.bar(c_125.index,c_125["country"])
plt.show()

In [ ]:
c_125.nunique()

In [ ]:
# Recheck how many cluster we should have as maximum
from sklearn.preprocessing import MinMaxScaler

df = data.drop("country", axis=1)
trim_df = df.drop(df.columns[:6], axis=1)
columns = list(trim_df.columns)
scaler = MinMaxScaler(feature_range=(0,1))
scale_df = scaler.fit_transform(trim_df)
scale_df = pd.DataFrame(scale_df, columns=columns)
scale_df.head()

In [ ]:
#Elbow Method 
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

kmeans = KMeans()
visualizer = KElbowVisualizer(kmeans, k=(2,15))
visualizer.fit(df)
visualizer.poof()

In [ ]:
# cluster analysis- Aysin

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5)

In [ ]:
kmeans.fit(scale_df)

In [ ]:
predicted_clusters = kmeans.predict(scale_df)

In [ ]:
centers = kmeans.cluster_centers_
labels = kmeans.labels_
labels

In [ ]:
len(labels)

In [ ]:
centers

In [ ]:
# scale_df = scale_df.add(labels)
scale_df['cluster'] = labels.tolist()
scale_df

In [ ]:
# analysis of the clusters- Michael

In [ ]:
scale_df

In [ ]:
wieghted_df = scale_df
wieghted_df

In [ ]:
wieghted_df['E2'] = 1.2 - wieghted_df['E2'] 

In [ ]:
wieghted_df['E4'] = 1.2 -wieghted_df['E4']

In [ ]:
wieghted_df['E6'] = 1.2 - wieghted_df['E6']

In [ ]:
wieghted_df['E8'] = 1.2 - wieghted_df['E8']

In [ ]:
wieghted_df['E10'] = 1.2 - wieghted_df['E10']

In [ ]:
wieghted_df['N2'] = 1.2 - wieghted_df['N2']

In [ ]:
wieghted_df['N4'] = 1.2 - wieghted_df['N4']

In [ ]:
wieghted_df['A1'] = 1.2 - wieghted_df['A1']

In [ ]:
wieghted_df['A3'] = 1.2 - wieghted_df['A3']

In [ ]:
wieghted_df['A5'] = 1.2- wieghted_df['A5']

In [ ]:
wieghted_df['A7'] = 1.2 - wieghted_df['A7']

In [ ]:
wieghted_df['C2'] = 1.2 - wieghted_df['C2']

In [ ]:
wieghted_df['C4'] = 1.2 - wieghted_df['C4']

In [ ]:
wieghted_df['C6'] = 1.2 - wieghted_df['C6']

In [ ]:
wieghted_df['C8'] = 1.2 - wieghted_df['C8']

In [ ]:
wieghted_df['O2'] = 1.2 - wieghted_df['O2']

In [ ]:
wieghted_df['O4'] = 1.2 - wieghted_df['O4']

In [ ]:
wieghted_df['O6'] = 1.2 - wieghted_df['O6']

In [ ]:
wieghted_df

In [ ]:
E = wieghted_df

In [ ]:
E['E_Sum_Result'] = wieghted_df.loc[:,["E1","E2","E3","E4","E5","E6","E7","E8","E9","E10"]].sum(axis=1)
E

In [ ]:
N = wieghted_df
N['N_Sum_Result'] = wieghted_df.loc[:,"N1":"N10"].sum(axis=1)
N

In [ ]:
A = wieghted_df
A['A_Sum_Result'] = wieghted_df.loc[:,"A1":"A10"].sum(axis=1)
A

In [ ]:
C = wieghted_df
C['C_Sum_Result'] = wieghted_df.loc[:,"C1":"C10"].sum(axis=1)
C

In [ ]:
O = wieghted_df
O['O_Sum_Result'] = wieghted_df.loc[:,"O1":"O10"].sum(axis=1)
O

In [ ]:
sums_df = pd.DataFrame()

In [ ]:
sums_df["O"] = O['O_Sum_Result']
sums_df["C"] = C['C_Sum_Result']
sums_df["E"] = E['E_Sum_Result']
sums_df["A"] = A['A_Sum_Result']
sums_df["N"] = N['N_Sum_Result']
sums_df["Cluster"] = wieghted_df['cluster']
sums_df

In [ ]:
cluster_avg = sums_df.groupby('Cluster').mean()
cluster_avg

In [ ]:
# analysis of the questions- Stephen

#Find silhouette score of current cluster model

from sklearn import metrics
from sklearn.metrics import pairwise_distances

raw_df = scale_df.drop(columns=["cluster"])

print(metrics.silhouette_score(raw_df, labels, metric='euclidean'))


In [ ]:
# Find the standard deviation of responses to each question

question_df = scale_df.groupby("cluster").mean()

question_std = question_df.std()

sorted_std = question_std.sort_values(ascending=False)

sorted_std

In [ ]:
#Plot standard deviation to find obvious discontinuities

sorted_std.plot.bar()

In [ ]:
print(sorted_std.index)

In [ ]:
#Create two subgroups of questions to analyze

cut1 = ['E7', 'N8', 'E5', 'N6', 'E3', 'N10', 'N7', 'E10', 'N1', 'E2', 'N9',
       'E9', 'E4', 'E1', 'E6', 'A7', 'C4', 'N5', 'A9', 'A4', 'N3', 'A2', 'A5',
       'E8', 'A6', 'C6', 'A10', 'N2', 'A8', 'A1', 'A3', 'C8', 'C5', 'C2', 'C9',
       'N4', 'C1']

cut2 = ['E7', 'N8', 'E5', 'N6', 'E3', 'N10', 'N7', 'E10', 'N1', 'E2', 'N9',
       'E9', 'E4', 'E1', 'E6', 'A7', 'C4', 'N5', 'A9', 'A4', 'N3']

brief1_df = raw_df[cut1]

brief2_df = raw_df[cut2]

In [ ]:
# Create cluster models for each subgroup

kmeans_brief1 = KMeans(n_clusters=5)
kmeans_brief1.fit(brief1_df)

kmeans_brief2 = KMeans(n_clusters=5)
kmeans_brief2.fit(brief2_df)


In [ ]:
#Compare silhouette scores for each subgroup to the original set

brief1_labels = kmeans_brief1.labels_
brief2_labels = kmeans_brief2.labels_

print(metrics.silhouette_score(raw_df, labels, metric='euclidean'))
print(metrics.silhouette_score(brief1_df, brief1_labels, metric='euclidean'))
print(metrics.silhouette_score(brief2_df, brief2_labels, metric='euclidean'))

In [ ]:
#Test each question one by one to identify useful questions

#Create data list

colsToDrop = ['E7', 'N8', 'E5', 'N6', 'E3', 'N10', 'N7', 'E10', 'N1', 'E2', 'N9',
       'E9', 'E4', 'E1', 'E6', 'A7', 'C4', 'N5', 'A9', 'A4', 'N3', 'A2', 'A5',
       'E8', 'A6', 'C6', 'A10', 'N2', 'A8', 'A1', 'A3', 'C8', 'C5', 'C2', 'C9',
       'N4', 'C1', 'O2', 'O10', 'O5', 'C7', 'C10', 'O7', 'C3', 'O1', 'O8',
       'O6', 'O4', 'O9', 'O3']

In [ ]:
#Check other cluster counts for better coverage with different question sets

clusterCount = [3,4,5,6,7,8,9,10]

#Establish base silhouette scores to use as thresholds for question analysis for each cluster

thresholds = []

for x in clusterCount:

    kmeans_control = KMeans(n_clusters=x)
    kmeans_control.fit(raw_df)

    control_labels = kmeans_control.labels_

    thresholds.append(metrics.silhouette_score(raw_df, control_labels, metric="euclidean"))

thresholds

In [ ]:
#establish the silhouette score threshold for each cluster

threshold_df = pd.DataFrame(clusterCount, columns=["Clusters"])
threshold_df["Threshold"] = thresholds

threshold_df

In [ ]:
# Run a new cluster analysis for each cluster count, omitting each question one at a time, and record the resulting silhouette score

scores_df = pd.DataFrame()
scores_df["Question"] = np.NaN
scores_df["Score"] = np.NaN
scores_df["N"] = np.NaN

clusterCount = [3,4,5,6,7,8,9,10]
counter = 0

for x in colsToDrop:
    test_df = raw_df.drop(columns=[x])
    
    for y in clusterCount:
        kmeans_test = KMeans(n_clusters=y)
        kmeans_test.fit(test_df)

        test_labels = kmeans_test.labels_

        score = metrics.silhouette_score(test_df, test_labels, metric="euclidean")

        newRow = pd.DataFrame({"Question": [x], "Score": [score], "N": [y]})
        scores_df = pd.concat([scores_df, newRow])
    
scores_df

In [ ]:
# For each cluster count, drop all questions where omitting the question resulted in a higher score.
# Re-run the cluster analysis using only qualifying questions.
# Visualize the silhouette scores for each number of clusters
# Save results to a dataframe

from yellowbrick.cluster import SilhouetteVisualizer

trim_df = pd.DataFrame()
trim_df["N"] = np.NaN
trim_df["Score"] = np.NaN
trim_df["Questions"] = np.NaN


for index, row in threshold_df.iterrows():
    temp_df = scores_df[scores_df["N"] == row[0]]
    temp_df = temp_df[temp_df["Score"] < row[1]]
    
    tempQuest = temp_df["Question"].tolist()
    
    tempTrim_df = raw_df[tempQuest]
    
    kmeans_temp = KMeans(n_clusters=int(row[0]))
    kmeans_temp.fit(tempTrim_df)

    temp_labels = kmeans_temp.labels_    

    trim_score = metrics.silhouette_score(tempTrim_df, temp_labels, metric="euclidean")
        
    newRow = pd.DataFrame({"N": row[0], "Score": [trim_score], "Questions": [tempQuest]})
    trim_df = pd.concat([trim_df, newRow])
    
    visualizer = SilhouetteVisualizer(kmeans_temp, colors='yellowbrick')
    
    visualizer.fit(tempTrim_df)
    visualizer.show()
    

trim_df

In [ ]:
# demographic analysis-James
#gender, age

In [ ]:
#demographic analysis -Hai
# country, handedness,engnat

In [ ]:
# country_df = scale_df
# country_df['country'] = data['country']
# country_df

In [ ]:
age_df = scale_df
age_df['age'] = data['age']
age_df

In [ ]:
#
#cluster0_df = age_df[age_df['cluster']==0]
# cluster0x_df = cluster0_df.loc[:, 'E1':'O10']
# cluster0y_df =  cluster0_df.loc[:, 'age']
#cluster0y_df.head()
#cluster0x_df.head()
#cluster0_df.head()

In [ ]:
cluster0_df = age_df[age_df['cluster']==0]
cluster0_corr = cluster0_df.corr()
cluster0_age=cluster0_corr.loc['age',:]

cluster1_df = age_df[age_df['cluster']==1]
cluster1_corr = cluster1_df.corr()
cluster1_age=cluster1_corr.loc['age',:]

cluster2_df = age_df[age_df['cluster']==2]
cluster2_corr = cluster2_df.corr()
cluster2_age=cluster2_corr.loc['age',:]

cluster3_df = age_df[age_df['cluster']==3]
cluster3_corr = cluster3_df.corr()
cluster3_age=cluster3_corr.loc['age',:]

cluster4_df = age_df[age_df['cluster']==4]
cluster4_corr = cluster4_df.corr()
cluster4_age=cluster4_corr.loc['age',:]

combine_age_df = pd.DataFrame({'cluster0':cluster0_age,'cluster1':cluster1_age,'cluster2':cluster2_age,'cluster3':cluster3_age,'cluster4':cluster4_age})
combine_age_df.head()

In [ ]:
combine_age_df = pd.DataFrame({'cluster0':cluster0_age,'cluster1':cluster1_age,'cluster2':cluster2_age,'cluster3':cluster3_age,'cluster4':cluster4_age})
combine_age_df.head()


In [ ]:
engnat_df = scale_df
engnat_df['engnat'] = data['engnat']
engnat_df=engnat_df.drop(['age'],axis =1)
engnat_df

In [ ]:
cluster0_df = engnat_df[engnat_df['cluster']==0]
cluster0_corr = cluster0_df.corr()
cluster0_engnat=cluster0_corr.loc['engnat',:]

cluster1_df = engnat_df[engnat_df['cluster']==1]
cluster1_corr = cluster1_df.corr()
cluster1_engnat=cluster1_corr.loc['engnat',:]

cluster2_df = engnat_df[engnat_df['cluster']==2]
cluster2_corr = cluster2_df.corr()
cluster2_engnat=cluster2_corr.loc['engnat',:]

cluster3_df = engnat_df[engnat_df['cluster']==3]
cluster3_corr = cluster3_df.corr()
cluster3_engnat=cluster3_corr.loc['engnat',:]

cluster4_df = engnat_df[engnat_df['cluster']==4]
cluster4_corr = cluster4_df.corr()
cluster4_engnat=cluster4_corr.loc['engnat',:]

combine_engnat_df = pd.DataFrame({'cluster0':cluster0_engnat,'cluster1':cluster1_engnat,'cluster2':cluster2_engnat,'cluster3':cluster3_engnat,'cluster4':cluster4_engnat})
combine_engnat_df.head()


In [ ]:

race_df = scale_df
race_df['race'] = data['race']
race_df=race_df.drop(['age','engnat'],axis =1)
race_df

In [ ]:
cluster0_df = race_df[race_df['cluster']==0]
cluster0_corr = cluster0_df.corr()
cluster0_race=cluster0_corr.loc['race',:]

cluster1_df = race_df[race_df['race']==1]
cluster1_corr = cluster1_df.corr()
cluster1_race=cluster1_corr.loc['race',:]

cluster2_df = race_df[race_df['cluster']==2]
cluster2_corr = cluster2_df.corr()
cluster2_race=cluster2_corr.loc['race',:]

cluster3_df = race_df[race_df['cluster']==3]
cluster3_corr = cluster3_df.corr()
cluster3_race=cluster3_corr.loc['race',:]

cluster4_df = race_df[race_df['cluster']==4]
cluster4_corr = cluster4_df.corr()
cluster4_race=cluster4_corr.loc['race',:]

combine_race_df = pd.DataFrame({'cluster0':cluster0_race,'cluster1':cluster1_race,'cluster2':cluster2_race,'cluster3':cluster3_race,'cluster4':cluster4_race})
combine_race_df.head()

In [ ]:
gender_df = scale_df
gender_df['gender'] = data['gender']
gender_df=gender_df.drop(['age','engnat','race'],axis =1)
gender_df

In [ ]:
cluster0_df = gender_df[gender_df['cluster']==0]
cluster0_corr = cluster0_df.corr()
cluster0_gender=cluster0_corr.loc['gender',:]

cluster1_df = gender_df[gender_df['cluster']==1]
cluster1_corr = cluster1_df.corr()
cluster1_gender=cluster1_corr.loc['gender',:]

cluster2_df = gender_df[gender_df['cluster']==2]
cluster2_corr = cluster2_df.corr()
cluster2_gender=cluster2_corr.loc['gender',:]

cluster3_df = gender_df[gender_df['cluster']==3]
cluster3_corr = cluster3_df.corr()
cluster3_gender=cluster3_corr.loc['gender',:]

cluster4_df = gender_df[gender_df['cluster']==4]
cluster4_corr = cluster4_df.corr()
cluster4_gender=cluster4_corr.loc['gender',:]
#cluster1_gender.head()
combine_gender_df = pd.DataFrame({'cluster0':cluster0_gender,'cluster1':cluster1_gender,'cluster2':cluster2_gender,'cluster3':cluster3_gender,'cluster4':cluster4_gender})
combine_gender_df.head()

In [ ]:
hand_df = scale_df
hand_df['hand'] = data['hand']
hand_df=hand_df.drop(['age','engnat','race','gender'],axis =1)
hand_df

In [ ]:
cluster0_df = hand_df[hand_df['cluster']==0]
cluster0_corr = cluster0_df.corr()
cluster0_hand=cluster0_corr.loc['hand',:]

cluster1_df = hand_df[hand_df['cluster']==1]
cluster1_corr = cluster1_df.corr()
cluster1_hand=cluster1_corr.loc['hand',:]

cluster2_df = hand_df[hand_df['cluster']==2]
cluster2_corr = cluster2_df.corr()
cluster2_hand=cluster2_corr.loc['hand',:]

cluster3_df = hand_df[hand_df['cluster']==3]
cluster3_corr = cluster3_df.corr()
cluster3_hand=cluster3_corr.loc['hand',:]

cluster4_df = hand_df[hand_df['cluster']==4]
cluster4_corr = cluster4_df.corr()
cluster4_hand=cluster4_corr.loc['hand',:]

combine_hand_df = pd.DataFrame({'cluster0':cluster0_hand,'cluster1':cluster1_hand,'cluster2':cluster2_hand,'cluster3':cluster3_hand,'cluster4':cluster4_hand})
combine_hand_df.head()